In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
df = pd.read_csv('Data_Preprocessed.csv')
df.shape

(283043, 33)

In [3]:
# нормализация из прошлого скрипта
from sklearn.preprocessing import StandardScaler, MinMaxScaler
normalize_columns = ['price', 'powerPS', 'model', 'brand', 'Age']
scaler = MinMaxScaler()
df[normalize_columns] = scaler.fit_transform(df[normalize_columns])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['kilometer']), df['kilometer'],
                                                    test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((226434, 32), (56609, 32))

In [5]:
# В качестве baseline возьмем наивный классификатор, его качество будет около 76%
df['kilometer'].value_counts(normalize=True)

1    0.762121
0    0.237879
Name: kilometer, dtype: float64

## Обучим нейронку

In [6]:
import tensorflow as tf
from tensorflow import keras

Задача нейронкой скорее всего не очень хорошо решается, так что ниже скрипты будут скорее чтобы показать, что так можно делать

#### Один слой

In [7]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

5661/5661 [==============================] - 4s 630us/step - loss: 0.3809 - accuracy: 0.8464 - val_loss: 0.3524 - val_accuracy: 0.8601
Epoch 2/10
5661/5661 [==============================] - 4s 637us/step - loss: 0.3480 - accuracy: 0.8614 - val_loss: 0.3464 - val_accuracy: 0.8609
Epoch 3/10
5661/5661 [==============================] - 4s 660us/step - loss: 0.3441 - accuracy: 0.8618 - val_loss: 0.3440 - val_accuracy: 0.8626
Epoch 4/10
5661/5661 [==============================] - 4s 697us/step 

#### Добавим слоев

In [9]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

5661/5661 [==============================] - 4s 738us/step - loss: 0.3652 - accuracy: 0.8515 - val_loss: 0.3619 - val_accuracy: 0.8511
Epoch 2/10
5661/5661 [==============================] - 4s 704us/step - loss: 0.3467 - accuracy: 0.8601 - val_loss: 0.3441 - val_accuracy: 0.8621
Epoch 3/10
5661/5661 [==============================] - 4s 690us/step - loss: 0.3423 - accuracy: 0.8619 - val_loss: 0.3420 - val_accuracy: 0.8611
Epoch 4/10
5661/5661 [==============================] - 4s 700us/step 

#### Попробуем по батчам

In [16]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=128)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1416/1416 [==============================] - 1s 980us/step - loss: 0.3696 - accuracy: 0.8500 - val_loss: 0.3495 - val_accuracy: 0.8602
Epoch 2/10
1416/1416 [==============================] - 1s 867us/step - loss: 0.3461 - accuracy: 0.8605 - val_loss: 0.3430 - val_accuracy: 0.8629
Epoch 3/10
1416/1416 [==============================] - 1s 984us/step - loss: 0.3421 - accuracy: 0.8618 - val_loss: 0.3458 - val_accuracy: 0.8609
Epoch 4/10
1416/1416 [==============================] - 1s 902us/step 

Вообще неплохо, давайте на другие метрики на тесте посмотрим, хотя без разницы сколько слоев, по батчам только быстрее обучать выходит, что логично, затем gpu и нужен

In [13]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score, recall_score

In [14]:
y_pred = tf.argmax(model.predict(X_test), 1)

In [15]:
accuracy = round(accuracy_score(y_test, y_pred), 2)
precision = round(precision_score(y_test, y_pred), 2)
recall = round(recall_score(y_test, y_pred), 2)
f1 = round(f1_score(y_test, y_pred), 2)
print('Test Scores')
print('Accuracy: {}, Precision: {}, Recall: {}, F1: {}'.format(accuracy, precision, recall, f1))

Test Scores
Accuracy: 0.86, Precision: 0.87, Recall: 0.96, F1: 0.92


В целом меня удовлетворяет полученное качество, с этим можно играться и дальше улучшать модель (в нейронках куда больше параметров чем в условном KNN), но не очень понятно зачем